<a href="https://colab.research.google.com/github/AlaxanderLee/Lecture-and-self-study/blob/master/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC(%EC%A0%95%EB%A6%AC%20%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 결정 트리